In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
import time
import csv
import re
from bs4 import BeautifulSoup
import requests

In [2]:
# csv파일 생성 및 쓰기
f = open("albaheaven.csv", "w")
wrt = csv.writer(f)
wrt.writerow(["근무지역", "기업명", "공고 제목 및 내용", "근무시간", "급여", "공고 등록일", "공고글URL","성별", "연령", "모집직종", "급여", "근무기간", "근무요일", "근무시간", "담당자명", "연락처", "지원방법", "지원마감일", "근무지주소"])

98

In [3]:
# 자동화된 Edge 창 실행
edge_driver = EdgeChromiumDriverManager().install()
service = Service(edge_driver)
driver = webdriver.Edge(service=service)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 7.86M/7.86M [00:01<00:00, 6.28MB/s]


In [4]:
# 알바천국 웹페이지 접속(광진구, 알바에 해당하는 데이터만 필터링)
url = "https://www.alba.co.kr/job/main.asp?WsSrchKeywordWord=&hidschContainText=&hidWsearchInOut=&hidSort=&hidSortOrder=&hidSortDate=&hidListView=LIST&hidSortCnt=50&hidSortFilter=N&hidJobKind=&hidJobKindMulti=&page=1&hidSearchyn=Y&strAreaMulti=02%7C%7C%EA%B4%91%EC%A7%84%EA%B5%AC%7C%7C%EC%A0%84%EC%B2%B4&schtext=&selGugun=%EA%B4%91%EC%A7%84%EA%B5%AC&selDong=%EC%A0%84%EC%B2%B4&hiretypecd=K01&lastschoolcd=&agelimit=&careercd=&searchterm=&hidCareerCD=&hidLastSchoolCD=&hidLastPayCD=&hidPayStart=&sortCnt=50"
driver.get(url)
time.sleep(2)

In [5]:
# 선택된 창이 꼬이지 않도록 초기화
driver.switch_to.window(driver.window_handles[0])

In [6]:
# 검색 화면 공고 리스트
# tbodys = driver.find_elements(By.XPATH, "/html/body/div[3]/form/div/div[2]/table/tbody")

In [7]:
# rows = []
# for tbody in tbodys:
#     rows.extend(tbody.find_elements(By.TAG_NAME, "tr"))


In [8]:
tbodys = driver.find_elements(By.XPATH, "/html/body/div[3]/form/div/div[2]/table/tbody")
rows = []
for tbody in tbodys:
    tr_elements = tbody.find_elements(By.TAG_NAME, "tr")
    filtered_tr_elements = [tr for tr in tr_elements if tr.get_attribute("class") != "summaryView"]
    rows.extend(filtered_tr_elements)
print(len(rows))

50


In [9]:

tbodys = driver.find_elements(By.XPATH, "/html/body/div[3]/form/div/div[2]/table/tbody")
rows = []
for tbody in tbodys:
    tr_elements = tbody.find_elements(By.TAG_NAME, "tr")
    filtered_tr_elements = [tr for tr in tr_elements if tr.get_attribute("class") != "summaryView"]
    rows.extend(filtered_tr_elements)
print(len(rows))


# In[173]:


for row in rows:
    link = row.find_element(By.CSS_SELECTOR, ".goodsList .title a")
    link_url = link.get_attribute("href")
   

    location = row.find_element(By.CLASS_NAME, "local").text
    company = row.find_element(By.CLASS_NAME, "company").text
    title = row.find_element(By.CLASS_NAME, "title").text
    time_info = row.find_element(By.CLASS_NAME, "data").text
    pay = row.find_element(By.CLASS_NAME, "pay").text
    reg_date = row.find_element(By.CLASS_NAME, "regDate").text

    
    # 링크에 접속
    driver.execute_script("window.open(arguments[0]);", link_url)
    time.sleep(2)
    
    # 새 탭으로 전환
    driver.switch_to.window(driver.window_handles[-1])
    
    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 상세 조건 리스트 선택
    condition_list = soup.find("div", class_="detail-content__condition-list")

    # 각 dl 태그에서 변수명과 변수값 가져오기
    details = {}
    dl_tags = condition_list.find_all("dl")
    for dl in dl_tags:
        dt = dl.find("dt").get_text(strip=True)
        if dl.find("dd")!=None:
            dd = dl.find("dd").get_text(strip=True)
        details[dt] = dd


    # 상세 조건 리스트 선택
    condition_list = soup.find("div", class_="detail-content__condition-list detail-content__list--last")

    # 각 dl 태그에서 변수명과 변수값 가져오기
    dl_tags = condition_list.find_all("dl")
    for dl in dl_tags:
        dt = dl.find("dt").get_text(strip=True)
        if dl.find("dd")!=None:
            dd = dl.find("dd").get_text(strip=True)
        details[dt] = dd

    # 상세 조건 리스트 선택
    condition_list = soup.find("div", class_="detail-content__recruit-list")

    # 각 dl 태그에서 변수명과 변수값 가져오기
    dl_tags = condition_list.find_all("dl")
    for dl in dl_tags:
        dt = dl.find("dt").get_text(strip=True)
        if dl.find("dd")!=None:
            dd = dl.find("dd").get_text(strip=True)
        details[dt] = dd
    
    # 지원방법
    elements = driver.find_elements(By.CSS_SELECTOR, ".detail-content__recruit--button a")
    apply = [element.text for element in elements]

    # 공고마감일
    deadline_element = driver.find_element(By.CLASS_NAME, "detail-content__recruit--date.itemDate")
    deadline = deadline_element.text
    
    
    # 근무지주소
    condition_element = driver.find_element(By.ID, "InfoWork")
    dl_tags = condition_element.find_elements(By.TAG_NAME, "dl")

    address = ""
    for dl in dl_tags:
        dt = dl.find_element(By.TAG_NAME, "dt")
        if dt.text == "근무지주소":
            dd = dl.find_element(By.TAG_NAME, "dd")
            address = dd.text
            break
    
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    wrt.writerow([location, company, title, time_info, pay, reg_date, link_url, details["성별"], details["연령"],details[ "모집직종"],details[ "급여"], details["근무기간"],details[ "근무요일"],details[ "근무시간"], details["담당자명"], details["연락처"], apply,deadline, address])


50


In [10]:
#공고 창 및 파일 닫기
driver.close()
f.close()